In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
model = ResNet50(weights='imagenet')

In [ ]:
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(img)

In [ ]:
x.shape

In [ ]:
plt.imshow(np.squeeze(x))

In [ ]:
preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]

In [ ]:
import tensorflow as tf; print('TensorFlow Version:', tf.__version__)
import keras; print('Keras Version:', keras.__version__)

In [ ]:
from keras.models import load_model
import keras.backend as K
from tensorflow.python.framework import graph_io
from tensorflow.python.tools import freeze_graph
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.training import saver as saver_lib

In [ ]:
K.set_learning_phase(0)

In [ ]:
sess = K.get_session()

In [ ]:
saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)

In [ ]:
checkpoint_path = saver.save(sess, 'saved_ckpt', global_step=0, latest_filename='checkpoint_state')

In [ ]:
graph_io.write_graph(sess.graph, '.', 'tmp.pb')

In [ ]:
in_names = model.input.op.name
print(in_names)

In [ ]:
out_names = model.output.op.name
print(out_names)

In [ ]:
freeze_graph.freeze_graph('./tmp.pb', '',
                          False, checkpoint_path, out_names,
                          "save/restore_all", "save/Const:0",
                          './frozen_graph.pb', False, "")

In [ ]:
import uff

In [ ]:
uff_model = uff.from_tensorflow_frozen_model('frozen_graph.pb', [out_names])

In [ ]:
!convert-to-uff frozen_graph.pb

In [ ]:
import tensorrt as trt


In [ ]:

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
model_file = "frozen_graph.uff"



In [ ]:
with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.UffParser() as parser:
    # Foo
    parser.register_input(in_names, trt.DimsCHW(3, 224, 224), trt.UffInputOrder.NCHW)
    parser.register_output(out_names)

In [ ]:
parser.parse(model_file, network)

In [ ]:
builder.max_workspace_size = 2**30

In [ ]:
engine = builder.build_cuda_engine(network)

In [2]:
# Import TensorFlow and TensorRT
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt





In [3]:
# Inference with TF-TRT frozen graph workflow:
graph = tf.Graph()

session = tf.Session()

In [4]:
with graph.as_default():
    with session:
        # First deserialize your frozen graph:
        with tf.gfile.GFile('frozen_graph.pb', 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())

        # Now you can create a TensorRT inference graph from your
        # frozen graph:
        trt_graph = trt.create_inference_graph(
            input_graph_def=graph_def,
            outputs=['fc1000/Softmax'])
        
        # Import the TensorRT graph into a new graph and run:
        output_node = tf.import_graph_def(
            trt_graph,
            return_elements=['fc1000/Softmax'])

INFO:tensorflow:Running against TensorRT version 0.0.0


In [6]:
output_node

[<tf.Operation 'import/fc1000/Softmax' type=Softmax>]

In [ ]:
'import/input_1'

In [7]:
with tf.Session() as sess:
    sess.run(output_node)

InvalidArgumentError: You must feed a value for placeholder tensor 'import/input_1' with dtype float and shape [?,224,224,3]
	 [[node import/input_1 (defined at <ipython-input-4-ea759e1b4134>:17) ]]

Caused by op 'import/input_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.5/asyncio/base_events.py", line 345, in run_forever

1048576